# Testing

(just load an input and watch the morphing)

In [ ]:
import neurite as ne
import numpy as np
from notebooks.setup import test_generator, model, latest_checkpoint

Eventually load the latest checkpoint

In [ ]:
model.load_weights(latest_checkpoint)

In [ ]:
model.evaluate(test_generator, return_dict=True)

Qualitative test for prostate deformation

In [ ]:
test_input, test_output, mr_targets, us_targets, _ = test_generator(10)
test_pred = model.predict(test_input)
# model.evaluate(test_input, test_output, return_dict=True)

In [ ]:
n_slice = 36
images = [img[0, :, :, n_slice * 2].squeeze() for img in [test_input[0], test_pred[0], test_input[1]]]
images += [img[0, :, :, n_slice].squeeze() for img in [test_input[2], test_pred[2], test_output[2]]]
titles = ['moving', 'moved', 'fixed', 'moving_seg', 'moved_seg', 'fixed_seg']
_ = ne.plot.slices(images, titles=titles, cmaps=['gray'], do_colorbars=True, grid=(2, 3), width=12)

In [ ]:
from skimage.transform import resize
import matplotlib.pyplot as plt

selector = 35

def compose_image(us_image, mr_image, us_seg, mr_seg, selector=50, four_by_four=False):
    # image
    n_slice_img = int(selector / 100.0 * us_image.shape[3])
    mr_image, us_image = [img[0, :, :, n_slice_img, 0] for img in [mr_image, us_image]]

    if four_by_four:
        overlapped = mr_image.copy()
        overlapped[0:40, 0:40] = us_image[0:40, 0:40]
        overlapped[0:40, 80:120] = us_image[0:40, 80:120]
        overlapped[40:80, 40:80] = us_image[40:80, 40:80]
        overlapped[40:80, 120:160] = us_image[40:80, 120:160]
        overlapped[80:120, 0:40] = us_image[80:120, 0:40]
        overlapped[80:120, 80:120] = us_image[80:120, 80:120]
        overlapped[120:160, 40:80] = us_image[120:160, 40:80]
        overlapped[120:160, 120:160] = us_image[120:160, 120:160]
    else:
        overlapped = mr_image.copy()
        overlapped[0:80, 0:80] = us_image[0:80, 0:80]
        overlapped[80:160, 80:160] = us_image[80:160, 80:160]

    # mask
    n_slice_mask = int(selector / 100 * us_seg.shape[3])
    mr_seg, us_seg = [resize(img[0, :, :, n_slice_mask, 0].astype(float), output_shape=(160, 160)) for img in [mr_seg, us_seg]]
    mask = np.zeros(shape=(us_seg.shape[0], us_seg.shape[1], 4))
    mask[:, :, 0] = us_seg
    mask[:, :, 2] = mr_seg
    mask = (mask > 0.5).astype(float)
    mask[:, :, 3] = np.bitwise_or(us_seg > 0.5, mr_seg > 0.5)

    return overlapped, mask, n_slice_img, n_slice_mask

overlapped, mask_seg, n_image_slice, _ = compose_image(test_output[0], test_pred[0], test_output[2], test_pred[2], selector)
_, mask_target, _, _ = compose_image(test_output[0], test_pred[0], us_targets[0][np.newaxis, ...], mr_targets[0][np.newaxis, ...], selector)

plt.figure(figsize=(8, 8))
plt.imshow(overlapped, cmap="gray", interpolation="nearest")
plt.imshow(mask_seg, alpha=0.20, interpolation='bicubic')
plt.imshow(mask_target, alpha=0.35, interpolation='bicubic')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(test_pred[0][0, ..., n_image_slice, 0], cmap="gray", interpolation="nearest")
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(test_output[0][0, ..., n_image_slice, 0], cmap="gray", interpolation="nearest")
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(test_input[0][0, ..., n_image_slice, 0], cmap="gray", interpolation="nearest")
plt.axis('off')
plt.show()

Plot flow (doesn't work, some kind of resampling is needed)

In [ ]:
flow = test_pred[1].squeeze()
_ = ne.plot.flow([flow[:, :, i, [0,1]] for i in range(0, 80, 8)], width=25)

In [ ]:
# import k3d
#
# flow = test_pred[1].squeeze()
#
# plot = k3d.plot()
# plot += k3d.factory.vector_field(flow)
# plot.display()